# Neural Network Implementation
We finally approach the neural network implementation of this paper. Now must make Train, Test Split. And find some metrics to measure on.

In [359]:
# Packages
import numpy as np
import pandas as pd
import torch
import scipy.stats as stats
import scipy.optimize as optim
import matplotlib.pyplot as plt
import time

import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler

# All Required DataSets
IVSurface = np.load('testSurface.pkl', allow_pickle = True)
SP500 = np.load('SP500.pkl', allow_pickle = True)
VIX = np.load('VIX.pkl', allow_pickle = True)

In [362]:
mergedData = pd.merge(pd.merge(IVSurface, SP500, how = 'left', on = 'Date'), VIX, how = 'left', on = 'Date')
mergedData = mergedData.filter(['strike', 'daysExp', 'fittedIV', 'Returns', 'VIX'])
mergedData['IVChange'] = mergedData.fittedIV.diff(periods = 441)
mergedData = mergedData.shift(periods = -441).dropna()
mergedData.daysExp = mergedData.daysExp/250
mergedData.strike = mergedData.strike ** (-1)
mergedData.to_pickle('finalDataSet.pkl')

## Training Data and Testing Data

In [402]:
trainSplit = 0.7
trainInt = int(trainSplit * mergedData.shape[0])
trainingData = mergedData.iloc[:trainInt,:]
testData = mergedData.iloc[trainInt:,:]

## Comparison Model (Hull-White)

In [403]:
'''Note that we use log-returns instead of simple returns due to the tractability of log returns. This differs from Hull-White'''
class HullWhiteModel():
    def __init__(self, a, b, c, returns, delta, timeToMaturity):
        self.a = a
        self.b = b
        self.c = c
        self.returns = returns
        self.delta = delta
        self.ttm = timeToMaturity
        
    def expected_change_delta(self):
        returns = self.returns
        quadraticTerm = self.a + self.b * self.delta + self.c * self.delta ** 2
        return returns * quadraticTerm / np.sqrt(self.ttm)
    
def hull_white_function(X, theta):
    moneyness = X[0,:]
    timeToMaturity = X[1,:]
    returns = X[2,:]
    
    quadraticTerm = theta[0] + theta[1] * moneyness + theta[2] * moneyness ** 2
    return returns * quadraticTerm / np.sqrt(timeToMaturity)

# Calibration Data
yData = np.array(trainingData.IVChange).T
xData = np.array(trainingData.filter(['strike', 'daysExp', 'Returns'])).T

initValue = [0, 0, 0]
# Parameterisation by OLS minimisation.
funcToMin = lambda theta: np.sum(hull_white_function(xData, theta) - yData)**2

hullParameters = optim.minimize(funcToMin, initValue).x

In [404]:
print(hullParameters)

[-0.08776002 -0.10180254 -0.12342559]


Sense check of the Hull-White parameters holds as volatility is increasing as time to maturity decreases, increasing with decreasing returns.

## Preparing Dataloader for PyTorch

In [405]:
class Factor_Model_Dataloader():
    def __init__(self, X, y, scale_data = True):
        #if scale_data:
            #X = StandardScaler().fit_transform(X)
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, i):
        return self.X[i], self.y[i]

## Creating 4 Factor Neural Network

In [420]:
class Four_Factor_Model(nn.Module):
    '''Multilayer Feed Forward Network for regression'''
    def __init__(self, hiddenNodes, dropout):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(4, hiddenNodes[0]),
            nn.Dropout(p = dropout),
            nn.ReLU(),
            nn.Linear(hiddenNodes[0], hiddenNodes[1]),
            nn.ReLU(),
            nn.Dropout(p = dropout),
            nn.Linear(hiddenNodes[1], hiddenNodes[2]),
            nn.ReLU(),
            nn.Linear(hiddenNodes[2], 1)
        )
        
    def forward(self, x):
        return self.layers(x)

## Preparing 4 Factor Dataset

In [421]:
[X, y] = [trainingData.filter(['strike', 'daysExp', 'Returns', 'VIX']).values,
          trainingData.filter(['IVChange']).values]

torchTrainingSet = Factor_Model_Dataloader(X, y)
trainLoader = torch.utils.data.DataLoader(torchTrainingSet, batch_size = 512, shuffle = True, num_workers = 0)

[XTest, yTest] = [testData.filter(['strike', 'daysExp', 'Returns', 'VIX']).values,
          testData.filter(['IVChange']).values]

torchTestSet = Factor_Model_Dataloader(XTest, yTest)
testLoader = torch.utils.data.DataLoader(torchTestSet, batch_size = testData.shape[0], shuffle = True, num_workers = 0)

## Choosing Loss Functions

In [422]:
four_factor_model = Four_Factor_Model([128, 64, 32], 0.25)
loss_func = nn.MSELoss()

## Training Four Factor Neural Network

In [423]:
epochs = 50
opt = torch.optim.Adam(four_factor_model.parameters(), lr = 1e-3)

for epoch in range(0, epochs):
    tic = time.time()
    currentLoss = 0.0
    for i, data in enumerate(trainLoader, 0):
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 1))
        
        opt.zero_grad()
        outputs = four_factor_model(inputs)
        loss = loss_func(outputs, targets)
        loss.backward()
        opt.step()
        currentLoss += loss.item()
        
    
    if (epoch + 1) % 10 == 0:
        print('Loss after Epoch %5d: %.5f' % (epoch+1, currentLoss))
        current_loss = 0.0
        toc = time.time()
        print('Training Time Epoch %5d: %.5f' % (epoch + 1, toc - tic))
        
print('Training finished')

Starting epoch 1
Starting epoch 2
Starting epoch 3
Starting epoch 4
Starting epoch 5
Starting epoch 6
Starting epoch 7
Starting epoch 8
Starting epoch 9
Starting epoch 10
Loss after Epoch    10: 3.58681
Training Time Epoch    10: 1.85598
Starting epoch 11
Starting epoch 12
Starting epoch 13
Starting epoch 14
Starting epoch 15
Starting epoch 16
Starting epoch 17
Starting epoch 18
Starting epoch 19
Starting epoch 20
Loss after Epoch    20: 3.58836
Training Time Epoch    20: 1.89653
Starting epoch 21
Starting epoch 22
Starting epoch 23
Starting epoch 24
Starting epoch 25
Starting epoch 26
Starting epoch 27
Starting epoch 28
Starting epoch 29
Starting epoch 30
Loss after Epoch    30: 3.58574
Training Time Epoch    30: 1.64387
Starting epoch 31
Starting epoch 32
Starting epoch 33
Starting epoch 34
Starting epoch 35
Starting epoch 36
Starting epoch 37
Starting epoch 38
Starting epoch 39
Starting epoch 40
Loss after Epoch    40: 3.58700
Training Time Epoch    40: 1.87834
Starting epoch 41
Sta

In [424]:
for i, data in enumerate(testLoader, 0):
    inputs, targets = data
    inputs, targets = inputs.float(), targets.float()
    targets = targets.reshape((targets.shape[0], 1))

nn4F = four_factor_model(inputs)
xEval = np.array(testData.filter(['strike', 'daysExp', 'Returns'])).T
hullWhitePred = hull_white_function(xEval, hullParameters)

# Preparing Three Factor Neural Network

In [397]:
[X, y] = [trainingData.filter(['strike', 'daysExp', 'Returns']).values,
          trainingData.filter(['IVChange']).values]

torchTrainingSet = Factor_Model_Dataloader(X, y)
trainLoader = torch.utils.data.DataLoader(torchTrainingSet, batch_size = 512, shuffle = True, num_workers = 0)

[XTest, yTest] = [testData.filter(['strike', 'daysExp', 'Returns']).values,
          testData.filter(['IVChange']).values]

torchTestSet = Factor_Model_Dataloader(XTest, yTest)
testLoader = torch.utils.data.DataLoader(torchTestSet, batch_size = testData.shape[0], shuffle = True, num_workers = 0)

In [398]:
class Three_Factor_Model(nn.Module):
    '''Multilayer Feed Forward Network for regression'''
    def __init__(self, hiddenNodes, dropout):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(3, hiddenNodes[0]),
            nn.Dropout(p = dropout),
            nn.ReLU(),
            nn.Linear(hiddenNodes[0], hiddenNodes[1]),
            nn.ReLU(),
            nn.Dropout(p = dropout),
            nn.Linear(hiddenNodes[1], hiddenNodes[2]),
            nn.ReLU(),
            nn.Linear(hiddenNodes[2], 1)
        )
        
    def forward(self, x):
        return self.layers(x)

In [399]:
three_factor_model = Three_Factor_Model([128, 64, 32], 0.25)
epochs = 50
opt = torch.optim.Adam(three_factor_model.parameters(), lr = 1e-3)

for epoch in range(0, epochs):
    tic = time.time()
    currentLoss = 0.0
    for i, data in enumerate(trainLoader, 0):
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 1))
        
        opt.zero_grad()
        outputs = three_factor_model(inputs)
        loss = loss_func(outputs, targets)
        loss.backward()
        opt.step()
        currentLoss += loss.item()
        
    if (epoch + 1) % 10 == 0:
        print('Loss after Epoch %5d: %.5f' % (epoch+1, currentLoss))
        current_loss = 0.0
        toc = time.time()
        print('Training Time Epoch %5d: %.5f' % (epoch + 1, toc - tic))
print('Training finished')

for i, data in enumerate(testLoader, 0):
    inputs, targets = data
    inputs, targets = inputs.float(), targets.float()
    targets = targets.reshape((targets.shape[0], 1))

nn3F = three_factor_model(inputs)

Starting epoch 1
Loss after Epoch     1: 3.62942
Training Time Epoch     1: 1.90744
Starting epoch 2
Loss after Epoch     2: 3.58940
Training Time Epoch     2: 1.58004
Starting epoch 3
Loss after Epoch     3: 3.58817
Training Time Epoch     3: 1.83430
Starting epoch 4
Loss after Epoch     4: 3.58573
Training Time Epoch     4: 1.60271
Starting epoch 5
Loss after Epoch     5: 3.58283
Training Time Epoch     5: 1.86800
Starting epoch 6
Loss after Epoch     6: 3.57423
Training Time Epoch     6: 1.60073
Starting epoch 7
Loss after Epoch     7: 3.56300
Training Time Epoch     7: 1.86253
Starting epoch 8
Loss after Epoch     8: 3.54430
Training Time Epoch     8: 1.60150
Starting epoch 9
Loss after Epoch     9: 3.53404
Training Time Epoch     9: 1.84258
Starting epoch 10
Loss after Epoch    10: 3.53115
Training Time Epoch    10: 1.61598
Starting epoch 11
Loss after Epoch    11: 3.52662
Training Time Epoch    11: 1.57334
Starting epoch 12
Loss after Epoch    12: 3.52558
Training Time Epoch    1

# Results

In [425]:
evaluations = np.stack([nn3F.detach().numpy()[:,0],nn4F.detach().numpy()[:,0], hullWhitePred, targets.detach().numpy()[:,0]], axis = 1)
evaluations = pd.DataFrame(evaluations, columns = ['3F_Model', '4F_Model', 'Hull_White', 'Targets'])

In [426]:
performance = [sum((evaluations['3F_Model'] - evaluations.Targets)**2), sum((evaluations['4F_Model'] - evaluations.Targets)**2), sum((evaluations.Hull_White - evaluations.Targets)**2)]
performance

[600.7064467989742, 599.45230774807, 601.1853788739089]

In [429]:
gain = [1 - performance[0]/performance[1], 1 - performance[0]/performance[2], 1 - performance[1]/performance[2]]
gain

[-0.002092141500990463, 0.000796646245508903, 0.0028827566117545222]